In [1]:
# https://www.kaggle.com/bryanb/fifa-player-stats-database?select=FIFA20_official_data.csv

import hsfs

connection = hsfs.connection()
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
0,application_1639038167363_0003,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

In [2]:
from hops import hdfs
from pyspark.sql.functions import lower, col, lit


defaware = ['22', '21', '20', '19', '18', '17']
for year in defaware:
    df=spark.read.format("csv").option("header","true").load(hdfs.project_path()+"Resources/FIFA" + year + "_official_data.csv")
    if year == "22" or year == "21" or year == "20":
        df = df.drop(col("DefensiveAwareness"))
        
    df = df.toDF(*[c.lower() for c in df.columns])
    df = df.toDF(*[c.replace(' ', '_') for c in df.columns])
        
    df.coalesce(1).write.mode("overwrite").csv(hdfs.project_path()+"Resources/FIFA" + year + "_official_data_cleaned.csv",header=True) 


In [3]:
from pyspark.sql.functions import *

# Fix 2017, which does not have a 'release_clause' column. Take the data from 2018
df1 = spark.read.format("csv").option("header","true").load(hdfs.project_path()+"Resources/FIFA17_official_data_cleaned.csv/")
df2 = spark.read.format("csv").option("header","true").load(hdfs.project_path()+"Resources/FIFA18_official_data_cleaned.csv/")

df1 = df1.alias('df1')
df2 = df2.alias('df2')

df1 = df1.join(df2, df1.id == df2.id, "left").select('df1.*', 'df2.release_clause')
df1 = df1.na.fill(value="€1.0M", subset=['release_clause'])

df1.show(n=2, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------
 id                       | 176580                                           
 name                     | L. Suárez                                        
 age                      | 29                                               
 photo                    | https://cdn.sofifa.com/players/176/580/17_60.png 
 nationality              | Uruguay                                          
 flag                     | https://cdn.sofifa.com/flags/uy.png              
 overall                  | 92                                               
 potential                | 92                                               
 club                     | FC Barcelona                                     
 club_logo                | https://cdn.sofifa.com/teams/241/light_30.png    
 value                    | €83M                                             
 wage                     | €525K                               

In [4]:
spark.catalog.clearCache()

In [ ]:
df1.coalesce(1).write.mode("overwrite").csv(hdfs.project_path()+"Resources/FIFA17_official_data_cleaned.csv",header=True)

In [7]:
hdfs.rmr(hdfs.project_path()+"Resources/FIFA17_official_data_cleaned.csv")
hdfs.move(hdfs.project_path()+"Resources/FIFA17_official_data_cleaned2.csv", hdfs.project_path()+"Resources/FIFA17_official_data_cleaned.csv")

In [8]:
df17 = spark.read.format("csv").option("header","true").load(hdfs.project_path()+"Resources/FIFA17_official_data_cleaned.csv/")

fg = fs.create_feature_group("fifa",
                        version=1,
                        description="Fifa players",
                        primary_key=['id'],
                        statistics_config=True,
                        online_enabled=True)
fg.save(df17)

In [ ]:
years = ['18', '19', '20', '21', '22']
for year in years:
    df=spark.read.format("csv").option("header","true").load(hdfs.project_path()+"Resources/FIFA" + year + "_official_data_cleaned.csv")
    fg.insert(df)

KeyboardInterrupt: 